In [1]:
import json, hashlib, base64
from mysql.connector import pooling

with open('../../mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
pool = pooling.MySQLConnectionPool(pool_name="mypool", pool_size=3, **config)

In [2]:
def get_board(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "SELECT b.bid, b.uid, b.title, b.content, b.modTime, b.viewCount, b.replyCount, b.files, u.uname \
        FROM board AS b JOIN users AS u ON b.uid=u.uid WHERE b.bid=%s"
    cur.execute(sql,(bid,))
    row = cur.fetchone()
    cur.close()
    conn.close()
    return row

In [3]:
b = get_board(1)
b

(1,
 'james',
 'test title',
 'contents',
 datetime.datetime(2023, 8, 16, 18, 13, 56),
 0,
 0,
 '{"list":[]}',
 '제임스')

In [3]:
field = 'title'
query = ''

In [4]:
def get_board_count(field, query):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "select count(bid) from board where isDeleted=0 AND %s like %s"
    cur.execute(sql,(field,f'%{query}%'))
    row = cur.fetchone()
    cur.close()
    conn.close()
    return row

In [9]:
a = get_board_count(field=field, query=query)[0]
a

12

In [6]:
def insert_board(params):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "insert into board values(default, %s, %s, %s, default, default, default, default, %s)"
    cur.execute(sql,params)
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [7]:
params = ('james','untitle','contents','{"list":[]}')

In [8]:
c = insert_board(params)

In [10]:
def update_board(params):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "update board set title=%s, content=%s, modTime=now(), files=%s where bid=%s"
    cur.execute(sql,params)
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [13]:
params = ('title','contents','{"list":[]}',9)

In [14]:
update_board(params)

In [15]:
def delete_board(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "UPDATE board SET isDeleted=1 WHERE bid=%s"
    cur.execute(sql,(bid,))
    row = cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return row

In [16]:
delete_board(1)

In [6]:
def increase_count(field,bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = f"UPDATE board SET {field}={field}+1 where bid = %s"
    cur.execute(sql,(bid,))
    cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()


In [7]:
increase_count(field='viewCount',bid=2)

In [8]:
def decrease_reply_count(bid):
    conn = pool.get_connection()
    cur = conn.cursor()
    sql = "UPDATE board SET replyCount=replyCOunt-1 where bid = %s"
    cur.execute(sql,(bid,))
    cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()

In [9]:
increase_count(field='replyCount',bid=2)

In [10]:
decrease_reply_count(2)